This tutorial is designed for one to explore the features of this denoising project. I am using Fastai library, which is built on top of PyTorch. There are several advantages to using Fastai. Ease of use, visualization of whether it be training and valid losses, graphs and images. You could technically run the code from a terminal and still be able to view the graphs and images via terminal. However, I am still in the process of figuring that out and Jupyter notebook will serve as another option until then.

The entire tutorial is broken down into following sections:
#### 1. Importing all the necessary libararies
#### 2. Converting the .nii.gz (nifti images of MRI) into two slices of tiff images
#### 3. Creating the Image Databunch 
#### 4. Creating the model (there are two options)
#### 5. Training and saving the model
    - finding the optimal learning rate
#### 6. Inferencing using the save model
#### 7. Calculating the pSNR and SSIM

# 1. Importing the stadard modules required
- Fastai,fastai.vision,fastai.callbacks,fastai.utils.mem: **To do all things Fastai**
- torch, torchvision, nn: **For working with PyTorch natively when needed**
- os,pathlib: **Deal with paths when working with various files**

In [ ]:
import os
import pdb
import fastai
import torch
import torchvision
from torch import nn
from fastai.vision import *
from fastai.callbacks import *
from fastai.utils.mem import *
from pathlib import Path

### Importing the modules and python functions I created
- fileconverter: **To convert 3D niftii files to 2D Tiff Images**
- cls_for_reading_tif: **A class to work with tiff files**
- create_data_bunch: **To generate dataLoaders to train the models**
- models: **A collection of models that one can choose from**
- create_Learner: **To train and save the train or intermediate models**
- preditor: **To run inference and save the result**

In [7]:
import file_converter
import cls_for_reading_tif as clsrt
import create_data_bunch as dataFuncs
import models as mymodels
import create_learner as CL
import predictor
import calculate_metrics

# 2. Converting the nifti files to tiff files
After this step, we will have 2D tiff images in train and target folders

### For the location of the 3d nifti image 
- Enter the location of the ground truth (zero noise)
- Enter the location of the image with noise
- Enter the location of target images
- Enter the location of the train images
- Enter the axis along which to take the slices of 2D images
- Enter the begin and end of the 3D we want to consider for 2D slice extraction

In [ ]:
zero_noise = "/Users/vishwanathsomashekar/Documents/Insight/mriDenoise/data/raw"
source = "/Users/vishwanathsomashekar/Documents/Insight/mriDenoise/data/preprocessed" #Enter the source directory here
dest_target = "/Users/vishwanathsomashekar/Documents/Insight/mriDenoise/data/processed/target" # Enter the destination directory here
dest_train = "/Users/vishwanathsomashekar/Documents/Insight/mriDenoise/data/processed/train"
axis = 1 # the direction along which we want to make the slices
startVol = 0.2 # all mri scans of the brain might also have some part of the neck. this start and end vol will make sure we have mostly the brain
endVolume = 0.8

### Call the function

In [ ]:
file_converter.conv_niftti_2_tif_noise_truth(source,zero_noise,dest_train,dest_target,axis,startVol,endVolume)

# 3. Creating Image Databunch for image size of 45 x 45 pixels
Since we don't have enough data to train the network, I have used progressive resizing. The full size image slice is 181 x 181 pixels. So, the idea is:
- to create data by resizing to 45 x 45 and train the network
- now replace the data with images that are resized to 90 x 90 pixels and further train the network
- finally use the full size 181 x 181 images to train it again. 

Note: all the while, we have not reinitialized the weights of the layers. we are starting from the model that was trained on the previous model size.

In [3]:
path_to_train = "/Users/vishwanathsomashekar/Documents/Insight/mriDenoise/data/processed/train"
path_to_target = "/Users/vishwanathsomashekar/Documents/Insight/mriDenoise/data/processed/target"
bs = 64
im_size = 45
data_sz_45= dataFuncs.generate_data_bunch(path_to_train,path_to_target,im_size,bs)

# 4. Creating the model to train

In [6]:
learner = CL.createLearner(data_sz_45,model_choice=2)

One can see the structure of the layers using the summary( ) method of the learner

In [ ]:
learner.summary()

# 5. Train the model

### Finding the optimal learning rate using lr_find( ) method

In [ ]:
learner.lr_find()

### You can plot the results of lr_find( ) using plot( ) method.
Choose a learning rate such that the chosen value is in the middle of the steepest slope. It is just an educated guess, and you may have to try different values based on your particular project

In [ ]:
learner.recorder.plot()

### Run the training based on learning rate and the num of epochs.
The code is written such that the best model during the run is save as **"best"** in the models directory in the folder **"train"**. You can change the name from "best" to anything you deem meaningful. This can be done in the **train_model method in create_learner.py** file. You can then used learner.load(best) method to restart traning

In [ ]:
# the first time you run this code, you will not have a best model available. Therefore, you will have to
# uncomment the code below and run that
#CL.train_model(learner,num_epochs=1,lr=1e-3)
CL.train_model(learner,num_epochs=1,lr=1e-3,cont_from_model="best")

### Save the model
Once you think you have trained enough this image size, save the model in order to use that to train at a larger image size of 90 x 90 pixels

In [ ]:
learner.save("a_meaningful_name")

### Now create a new databunch like we did before, but this time for a image size of 90 x 90 pixels
A general rule of thumb, in order to fit about the same number of images on the GPU, if you doubled the image size, you will need to halve the batch size. I increased the im_size to 90 x 90 and decreased the batch size from 64 to 32

In [ ]:
bs = 32
im_size = 90
data_sz_90= dataFuncs.generate_data_bunch(path_to_train,path_to_target,im_size,bs)

### Load this new databunch into the learner we created earlier

In [ ]:
learner.data = data_sz_90

### Now train again like before taking the advantae of lr_find and lr.recorder.plot to find an optimal learning rate.

In [ ]:
learner.lr_find()
learner.recorder.plot()
# the first time you run this code, you will not have a best model available. Therefore, you will have to
# uncomment the code below and run that
#CL.train_model(learner,num_epochs=1,lr=1e-3)
CL.train_model(learner,num_epochs=1,lr=1e-3,cont_from_model="best")

After training enough, save the model with a meaningful name. using learner.save("a_meaningful_name")

### Follow the same procedure as before and resize the image, this time to 181 x 181. Again, a good rule of thumb is to halve the bs everytime we make the image size twice as big.

In [ ]:
bs = 16
im_size = 181
data_sz_181= dataFuncs.generate_data_bunch(path_to_train,path_to_target,im_size,bs)

### Replace data in the learner and train like before using the help of lr_find and lr.recorder.plot()

In [ ]:
learner.data = data_sz_181

In [ ]:
learner.lr_find()
learner.recorder.plot()
# the first time you run this code, you will not have a best model available. Therefore, you will have to
# uncomment the code below and run that
#CL.train_model(learner,num_epochs=1,lr=1e-3)
CL.train_model(learner,num_epochs=1,lr=1e-3,cont_from_model="best")

### Finally when you have done enough training, export the model

In [ ]:
learner.export()

# Inferencing
Use the predictor.predict(filename) method to inference.


In [ ]:
# uncomment the below line and change the path to your file. or if you have a lot of images to inference upon,
# you can use the predict method in a simple for or while loop. The method will also save a tif file in the same
# location as the file with a _predict appended to the file name
# filename = "/Users/vishwanathsomashekar/Documents/Insight/mriDenoise/data/processed/train/t1_icbm_normal_1mm_pn1_rf0_48.tif"
predictor.predict(filename)

## Displaying the input and the predicted images

In [ ]:
# im = os.path.join(root_dir, "train","t1_icbm_normal_1mm_pn3_rf0_50.tif" )#"/content/gdrive/My Drive/data/train/t1_icbm_normal_1mm_pn3_rf0_50.tif"
im_input = "/home/ubuntu/mriDenoise/data/processed/train/t1_icbm_normal_1mm_pn9_rf0_50.tif"

In [ ]:
im1 = open_tiff(im_input)
im1.show(cmap='gray', figsize=(10,10))

In [ ]:
# im = os.path.join(root_dir, "train","t1_icbm_normal_1mm_pn3_rf0_50.tif" )#"/content/gdrive/My Drive/data/train/t1_icbm_normal_1mm_pn3_rf0_50.tif"
im_predicted = "/home/ubuntu/mriDenoise/data/processed/train/t1_icbm_normal_1mm_pn9_rf0_50_predict.tif"

In [ ]:
im2 = open_tiff(im_predicted)
im2.show(cmap='gray', figsize=(10,10))

# Calculating metrics

In [ ]:
psnr_before,ssim_ = calculate_metrics(im_zero_noise,im1)